In [1]:
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/allenai/ValuePrism

fatal: destination path 'ValuePrism' already exists and is not an empty directory.


In [2]:
import pandas as pd, numpy as np, os, json, tqdm

In [3]:
full = pd.read_csv('ValuePrism/full/full.csv')
full.head()

,situation,vrd,text,explanation,valence
0,Pulling a lever that diverts a trolley away fr...,Value,Preservation of life,Saving the group of people from harm by divert...,Supports
1,Pulling a lever that diverts a trolley away fr...,Value,Minimizing harm,Diverting the trolley reduces overall harm by ...,Supports
2,Pulling a lever that diverts a trolley away fr...,Value,Autonomy,The person who was not initially in danger is ...,Opposes
3,Pulling a lever that diverts a trolley away fr...,Right,Right to life,The group of people tied to the tracks have a ...,Supports
4,Pulling a lever that diverts a trolley away fr...,Right,Right to safety,The person who would be killed by the diverted...,Opposes


In [8]:
example = full.sample(1).iloc[0]
print(example['situation'])
print(example['explanation'])

Speeding but not hitting those children
As long as they are not breaking the law or endangering others, individuals have the right to move at their own pace on roads and highways. Speeding may infringe on this right for others by creating an unsafe environment.


### How to generate a perspective?

We need to generate a perspective for each question / answer pair. To keep it simple, we'll just use the question and explanation text with some labels.

In [7]:
full['perspective'] = "Value: "+full['text']+"\nExplanation: "+full['explanation']

In [20]:
full

,situation,vrd,text,explanation,valence,perspective
0,Pulling a lever that diverts a trolley away fr...,Value,Preservation of life,Saving the group of people from harm by divert...,Supports,Value: Preservation of life\nExplanation: Savi...
1,Pulling a lever that diverts a trolley away fr...,Value,Minimizing harm,Diverting the trolley reduces overall harm by ...,Supports,Value: Minimizing harm\nExplanation: Diverting...
2,Pulling a lever that diverts a trolley away fr...,Value,Autonomy,The person who was not initially in danger is ...,Opposes,Value: Autonomy\nExplanation: The person who w...
3,Pulling a lever that diverts a trolley away fr...,Right,Right to life,The group of people tied to the tracks have a ...,Supports,Value: Right to life\nExplanation: The group o...
4,Pulling a lever that diverts a trolley away fr...,Right,Right to safety,The person who would be killed by the diverted...,Opposes,Value: Right to safety\nExplanation: The perso...
...,...,...,...,...,...,...
218403,Giving your life to save others.,Value,Self-preservation,"By giving up your life, you forsake the value ...",Opposes,Value: Self-preservation\nExplanation: By givi...
218404,Giving your life to save others.,Right,Right to life,"People have a fundamental right to life, and b...",Supports,Value: Right to life\nExplanation: People have...
218405,Giving your life to save others.,Right,Right to autonomy,"By giving up your life, you could argue that y...",Opposes,Value: Right to autonomy\nExplanation: By givi...
218406,Giving your life to save others.,Duty,Duty to protect,"In acknowledging that others are in danger, yo...",Supports,Value: Duty to protect\nExplanation: In acknow...


In [13]:
value_perspectives = full.groupby('situation')['perspective'].apply(lambda x: x.to_list())

In [19]:
full['vrd'].unique()

array(['Value', 'Right', 'Duty'], dtype=object)

In [16]:
value_perspectives.to_frame().reset_index()

,situation,perspective
0,"""Taking care"" of Gigi",[Value: Well-being\nExplanation: Providing car...
1,"""Taking care"" of Gigi (my friend)",[Value: Friendship\nExplanation: Valuing the b...
2,"""Told your friends about your bonuses""",[Value: Honesty\nExplanation: Sharing truthful...
3,"""borrowing"" somebody's credit card",[Value: Respect for others' property\nExplanat...
4,"""borrowing"" the nerds' lunch money to ""give to...",[Value: Justice\nExplanation: Taking money fro...
...,...,...
31023,your boss fires you,[Value: Fairness\nExplanation: If the boss has...
31024,zach defeating mary,[Value: Fair competition\nExplanation: Ensurin...
31025,zach defeating sally,[Value: Fair competition\nExplanation: If the ...
31026,zach winning over dan,[Value: Fairness\nExplanation: Fairness in com...


In [21]:
value_perspectives.to_frame().reset_index().to_csv('data/ValuePrism_perspectives.csv')